# 📊 ANÁLISIS ESTADÍSTICO - TIENDA AURELION

**Sprint 2 - Introducción a la Inteligencia Artificial - IBM**

**Autor:** Martos Ludmila  
**DNI:** 34811650

---

## 📋 Índice

1. [Carga de Datos](#carga)
2. [Estadísticas Descriptivas Básicas](#descriptivas)
3. [Identificación de Distribuciones](#distribuciones)
4. [Análisis de Correlaciones](#correlaciones)
5. [Detección de Outliers](#outliers)
6. [Gráficos Representativos](#graficos)
7. [Resumen Ejecutivo](#resumen)


---

## 📊 Dashboard Power BI - Sprint 4

Los mismos datos analizados en este notebook se utilizan también en un **dashboard en Power BI**:

- Archivo principal: `Power BI/Sprint4.pbix`
- Imagen del dashboard: `Power BI/dashboard.jpg`
- Documentación detallada en: `Power BI/Documentacion_Sprint4.md`

> Sugerencia: puedes mostrar primero esta imagen del dashboard en una presentación y luego profundizar en los análisis de este notebook.



---

## 1. Carga de Datos <a id='carga'></a>

Primero cargamos todas las librerías necesarias y los datos de los 4 archivos CSV.


In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest, shapiro
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Librerías importadas correctamente")


In [ ]:
# Detectar rutas de archivos
def obtener_rutas_csv():
    """Obtiene las rutas correctas de los CSVs."""
    # Para Jupyter Notebook, buscamos desde la carpeta programas
    base_dir = os.path.join("..", "datos")
    return {
        'productos': os.path.join(base_dir, "productos.csv"),
        'clientes': os.path.join(base_dir, "clientes.csv"),
        'ventas': os.path.join(base_dir, "ventas.csv"),
        'detalle_ventas': os.path.join(base_dir, "detalle_ventas.csv")
    }

ARCHIVOS_CSV = obtener_rutas_csv()
print("📁 Rutas de archivos configuradas")


In [ ]:
# Cargar datos
print("=" * 80)
print("CARGANDO DATOS...")
print("=" * 80)

df_productos = pd.read_csv(ARCHIVOS_CSV['productos'], encoding='utf-8')
df_clientes = pd.read_csv(ARCHIVOS_CSV['clientes'], encoding='utf-8')
df_ventas = pd.read_csv(ARCHIVOS_CSV['ventas'], encoding='utf-8')
df_detalle = pd.read_csv(ARCHIVOS_CSV['detalle_ventas'], encoding='utf-8')

# Preparar datos combinados para análisis
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'])
df_completo = df_detalle.merge(df_productos, left_on='id_producto', right_on='id', how='left')
df_completo = df_completo.merge(df_ventas, left_on='id_venta', right_on='id_venta', how='left')
df_completo = df_completo.merge(df_clientes, left_on='id_cliente', right_on='id', how='left')

print(f"✅ Productos cargados: {len(df_productos)}")
print(f"✅ Clientes cargados: {len(df_clientes)}")
print(f"✅ Ventas cargadas: {len(df_ventas)}")
print(f"✅ Detalles de ventas: {len(df_detalle)}")
print()


In [ ]:
# Vista previa de los datos
print("📊 Vista previa de Productos:")
display(df_productos.head())

print("\n📊 Vista previa de Ventas:")
display(df_ventas.head())


---

## 2. Estadísticas Descriptivas Básicas <a id='descriptivas'></a>

Calculamos estadísticas descriptivas para las variables numéricas principales.


In [ ]:
print("=" * 80)
print("1. ESTADÍSTICAS DESCRIPTIVAS BÁSICAS")
print("=" * 80)

print("\n📊 ESTADÍSTICAS DESCRIPTIVAS - PRODUCTOS")
print("-" * 80)
stats_productos = df_productos[['precio', 'stock']].describe()
display(stats_productos)


In [ ]:
print("\n📈 Información adicional - Productos:")
print(f"   - Mediana de precio: {df_productos['precio'].median():.2f}")
print(f"   - Mediana de stock: {df_productos['stock'].median():.2f}")
print(f"   - Desviación estándar precio: {df_productos['precio'].std():.2f}")
print(f"   - Desviación estándar stock: {df_productos['stock'].std():.2f}")
print(f"   - Rango precio: {df_productos['precio'].max() - df_productos['precio'].min()}")
print(f"   - Rango stock: {df_productos['stock'].max() - df_productos['stock'].min()}")


In [ ]:
print("\n📊 ESTADÍSTICAS DESCRIPTIVAS - VENTAS")
print("-" * 80)
stats_ventas = df_ventas[['total']].describe()
display(stats_ventas)
print(f"\n   - Mediana: {df_ventas['total'].median():.2f}")
print(f"   - Desviación estándar: {df_ventas['total'].std():.2f}")


In [ ]:
print("\n📊 ESTADÍSTICAS DESCRIPTIVAS - DETALLE DE VENTAS")
print("-" * 80)
stats_detalle = df_detalle[['cantidad', 'precio_unitario', 'subtotal']].describe()
display(stats_detalle)


---

## 3. Identificación del Tipo de Distribución <a id='distribuciones'></a>

Analizamos la distribución de las variables principales para identificar si siguen una distribución normal o tienen sesgo.


In [ ]:
def analizar_distribucion(data, nombre_var):
    """Analiza la distribución de una variable."""
    print(f"\n📈 Análisis de distribución: {nombre_var}")
    print("-" * 80)
    
    # Estadísticas de forma
    skewness = stats.skew(data.dropna())
    kurtosis = stats.kurtosis(data.dropna())
    
    print(f"   Asimetría (Skewness): {skewness:.4f}")
    print(f"   Curtosis (Kurtosis): {kurtosis:.4f}")
    
    # Test de normalidad
    if len(data.dropna()) <= 50:
        stat, p_value = shapiro(data.dropna())
        test_name = "Shapiro-Wilk"
    else:
        stat, p_value = normaltest(data.dropna())
        test_name = "D'Agostino-Pearson"
    
    print(f"\n   Test de normalidad ({test_name}):")
    print(f"   - Estadístico: {stat:.4f}")
    print(f"   - p-value: {p_value:.4f}")
    
    # Interpretación
    if p_value > 0.05:
        print(f"   ✅ Los datos siguen una distribución normal (p > 0.05)")
        tipo_dist = "Normal"
    elif skewness > 1:
        print(f"   ⚠️  Distribución asimétrica positiva (sesgada a la derecha)")
        tipo_dist = "Asimétrica Positiva"
    elif skewness < -1:
        print(f"   ⚠️  Distribución asimétrica negativa (sesgada a la izquierda)")
        tipo_dist = "Asimétrica Negativa"
    else:
        print(f"   ⚠️  Distribución no normal")
        tipo_dist = "No Normal"
    
    return tipo_dist, skewness, kurtosis

print("=" * 80)
print("2. IDENTIFICACIÓN DEL TIPO DE DISTRIBUCIÓN DE VARIABLES")
print("=" * 80)


In [ ]:
distribuciones = {}
distribuciones['precio'], skew_precio, kurt_precio = analizar_distribucion(df_productos['precio'], 'Precio de Productos')
distribuciones['stock'], skew_stock, kurt_stock = analizar_distribucion(df_productos['stock'], 'Stock de Productos')
distribuciones['total'], skew_total, kurt_total = analizar_distribucion(df_ventas['total'], 'Total de Ventas')


---

## 4. Análisis de Correlaciones <a id='correlaciones'></a>

Examinamos las correlaciones entre las variables principales para identificar relaciones significativas.


In [ ]:
print("=" * 80)
print("3. ANÁLISIS DE CORRELACIONES ENTRE VARIABLES PRINCIPALES")
print("=" * 80)

print("\n📊 Matriz de Correlación - Variables de Productos")
print("-" * 80)
corr_productos = df_productos[['precio', 'stock']].corr()
display(corr_productos)
print(f"\n   Correlación Precio-Stock: {corr_productos.loc['precio', 'stock']:.4f}")


In [ ]:
print("\n📊 Matriz de Correlación - Variables de Ventas")
print("-" * 80)
corr_ventas = df_detalle[['cantidad', 'precio_unitario', 'subtotal']].corr()
display(corr_ventas)


In [ ]:
# Correlación entre precio de producto y cantidad vendida
print("\n📊 Correlación Precio vs Cantidad Vendida")
print("-" * 80)
df_precio_cantidad = df_completo.groupby('id_producto').agg({
    'precio': 'first',
    'cantidad': 'sum'
}).reset_index()
corr_precio_cantidad = df_precio_cantidad[['precio', 'cantidad']].corr()
display(corr_precio_cantidad)
print(f"\n   Correlación Precio-Cantidad Vendida: {corr_precio_cantidad.loc['precio', 'cantidad']:.4f}")


In [ ]:
# Interpretación de correlaciones
print("\n💡 Interpretación de Correlaciones:")
print("-" * 80)
corr_precio_stock = corr_productos.loc['precio', 'stock']
if abs(corr_precio_stock) < 0.3:
    print("   - Precio y Stock: Correlación débil o inexistente")
elif abs(corr_precio_stock) < 0.7:
    print("   - Precio y Stock: Correlación moderada")
else:
    print("   - Precio y Stock: Correlación fuerte")

corr_precio_cant = corr_precio_cantidad.loc['precio', 'cantidad']
if corr_precio_cant < -0.3:
    print("   - Precio y Cantidad Vendida: Correlación negativa (productos más caros se venden menos)")
elif corr_precio_cant > 0.3:
    print("   - Precio y Cantidad Vendida: Correlación positiva (productos más caros se venden más)")
else:
    print("   - Precio y Cantidad Vendida: Correlación débil (el precio no influye mucho en la demanda)")


---

## 5. Detección de Outliers <a id='outliers'></a>

Utilizamos el método IQR (Interquartile Range) para identificar valores extremos.


In [ ]:
def detectar_outliers_iqr(data, nombre_var):
    """Detecta outliers usando el método IQR."""
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    
    print(f"\n📊 Detección de Outliers: {nombre_var}")
    print("-" * 80)
    print(f"   Q1 (25%): {Q1:.2f}")
    print(f"   Q3 (75%): {Q3:.2f}")
    print(f"   IQR: {IQR:.2f}")
    print(f"   Límite inferior: {lower_bound:.2f}")
    print(f"   Límite superior: {upper_bound:.2f}")
    print(f"   Outliers detectados: {len(outliers)}")
    
    if len(outliers) > 0:
        print(f"   Valores outliers:")
        for idx, val in outliers.items():
            print(f"     - {val:.2f}")
    else:
        print(f"   ✅ No se detectaron outliers")
    
    return outliers

print("=" * 80)
print("4. DETECCIÓN DE OUTLIERS (VALORES EXTREMOS)")
print("=" * 80)


In [ ]:
outliers_precio = detectar_outliers_iqr(df_productos['precio'], 'Precio de Productos')
outliers_stock = detectar_outliers_iqr(df_productos['stock'], 'Stock de Productos')
outliers_total = detectar_outliers_iqr(df_ventas['total'], 'Total de Ventas')


---

## 6. Gráficos Representativos <a id='graficos'></a>

Generamos gráficos profesionales para visualizar los datos y los resultados del análisis.

### Gráfico 1: Distribución de Precios


In [ ]:
# Crear carpeta para gráficos si no existe
output_dir = os.path.join("..", "graficos")
os.makedirs(output_dir, exist_ok=True)

print("=" * 80)
print("5. GENERANDO GRÁFICOS REPRESENTATIVOS")
print("=" * 80)


### Gráfico 2: Matriz de Correlación


In [ ]:
# GRÁFICO 1: Distribución de Precios con Histograma y Box Plot
print("\n📊 Generando Gráfico 1: Distribución de Precios...")
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Histograma de precios
axes[0].hist(df_productos['precio'], bins=15, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].axvline(df_productos['precio'].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df_productos["precio"].mean():.2f}')
axes[0].axvline(df_productos['precio'].median(), color='green', linestyle='--', linewidth=2, label=f'Mediana: {df_productos["precio"].median():.2f}')
axes[0].set_xlabel('Precio (monedas de oro)', fontsize=12)
axes[0].set_ylabel('Frecuencia', fontsize=12)
axes[0].set_title('Distribución de Precios de Productos', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot de precios
box_data = [df_productos['precio']]
axes[1].boxplot(box_data, labels=['Precio'])
axes[1].set_ylabel('Precio (monedas de oro)', fontsize=12)
axes[1].set_title('Box Plot - Precios de Productos', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'grafico1_distribucion_precios.png'), dpi=300, bbox_inches='tight')
print("   ✅ Guardado: grafico1_distribucion_precios.png")
plt.show()


### Gráfico 3: Análisis de Outliers y Tendencias de Ventas


In [ ]:
# GRÁFICO 2: Correlación entre Variables Principales
print("\n📊 Generando Gráfico 2: Matriz de Correlación...")
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Matriz de correlación productos
corr_mat_productos = df_productos[['precio', 'stock']].corr()
sns.heatmap(corr_mat_productos, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=2, ax=axes[0], cbar_kws={"shrink": 0.8})
axes[0].set_title('Correlación: Precio vs Stock', fontsize=14, fontweight='bold')

# Matriz de correlación ventas
corr_mat_ventas = df_detalle[['cantidad', 'precio_unitario', 'subtotal']].corr()
sns.heatmap(corr_mat_ventas, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=2, ax=axes[1], cbar_kws={"shrink": 0.8})
axes[1].set_title('Correlación: Variables de Ventas', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'grafico2_matriz_correlacion.png'), dpi=300, bbox_inches='tight')
print("   ✅ Guardado: grafico2_matriz_correlacion.png")
plt.show()


### Gráfico 3: Análisis de Outliers y Tendencias de Ventas

In [ ]:
# GRÁFICO 3: Análisis de Outliers y Tendencias de Ventas
print("\n📊 Generando Gráfico 3: Análisis de Outliers y Ventas...")
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Scatter plot Precio vs Stock con outliers marcados
axes[0, 0].scatter(df_productos['precio'], df_productos['stock'], alpha=0.6, s=100, color='steelblue')
# Marcar outliers de precio
if len(outliers_precio) > 0:
    outliers_df = df_productos[df_productos['precio'].isin(outliers_precio)]
    axes[0, 0].scatter(outliers_df['precio'], outliers_df['stock'], color='red', s=150, marker='x', 
                       label='Outliers', linewidths=3)
axes[0, 0].set_xlabel('Precio (monedas)', fontsize=11)
axes[0, 0].set_ylabel('Stock (unidades)', fontsize=11)
axes[0, 0].set_title('Precio vs Stock (Outliers marcados)', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Box plot de total de ventas
axes[0, 1].boxplot([df_ventas['total']], labels=['Total Ventas'])
axes[0, 1].set_ylabel('Total (monedas)', fontsize=11)
axes[0, 1].set_title('Distribución de Totales de Ventas', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Ventas por fecha
df_ventas_fecha = df_ventas.groupby(df_ventas['fecha'].dt.date)['total'].sum().reset_index()
df_ventas_fecha['fecha'] = pd.to_datetime(df_ventas_fecha['fecha'])
axes[1, 0].plot(df_ventas_fecha['fecha'], df_ventas_fecha['total'], marker='o', linewidth=2, markersize=8, color='green')
axes[1, 0].set_xlabel('Fecha', fontsize=11)
axes[1, 0].set_ylabel('Total Ventas (monedas)', fontsize=11)
axes[1, 0].set_title('Evolución de Ventas por Fecha', fontsize=12, fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# Top 5 productos más vendidos
productos_vendidos = df_detalle.groupby('id_producto')['cantidad'].sum().sort_values(ascending=False).head(5)
productos_top = df_productos[df_productos['id'].isin(productos_vendidos.index)][['nombre', 'id']]
productos_top = productos_top.merge(productos_vendidos.reset_index(), left_on='id', right_on='id_producto')
axes[1, 1].barh(productos_top['nombre'], productos_top['cantidad'], color='coral')
axes[1, 1].set_xlabel('Cantidad Vendida', fontsize=11)
axes[1, 1].set_title('Top 5 Productos Más Vendidos', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'grafico3_outliers_ventas.png'), dpi=300, bbox_inches='tight')
print("   ✅ Guardado: grafico3_outliers_ventas.png")
plt.show()

print("\n✅ Todos los gráficos generados exitosamente en la carpeta 'graficos/'")


---

## 7. Resumen Ejecutivo <a id='resumen'></a>

Resumen completo de los hallazgos del análisis estadístico.


In [ ]:
print("=" * 80)
print("RESUMEN EJECUTIVO DEL ANÁLISIS")
print("=" * 80)

print("\n📊 VARIABLES ANALIZADAS:")
print(f"   - Precio de productos: {len(df_productos)} registros")
print(f"   - Stock de productos: {len(df_productos)} registros")
print(f"   - Total de ventas: {len(df_ventas)} registros")
print(f"   - Detalles de ventas: {len(df_detalle)} registros")

print("\n📈 DISTRIBUCIONES IDENTIFICADAS:")
for var, tipo in distribuciones.items():
    print(f"   - {var.capitalize()}: {tipo}")

print("\n🔗 CORRELACIONES PRINCIPALES:")
print(f"   - Precio vs Stock: {corr_productos.loc['precio', 'stock']:.4f}")
print(f"   - Precio vs Cantidad Vendida: {corr_precio_cantidad.loc['precio', 'cantidad']:.4f}")

print("\n⚠️  OUTLIERS DETECTADOS:")
print(f"   - Precio: {len(outliers_precio)} valores extremos")
print(f"   - Stock: {len(outliers_stock)} valores extremos")
print(f"   - Total Ventas: {len(outliers_total)} valores extremos")

print("\n" + "=" * 80)
print("ANÁLISIS COMPLETADO EXITOSAMENTE")
print("=" * 80)


---

## 💡 Visualización Interactiva

Los datos de este análisis también están disponibles en plataformas interactivas:

### 📊 Dashboard Power BI Desktop

Accede a un dashboard profesional con visualizaciones interactivas y filtros dinámicos:
- **Ubicación**: `Sprint-2/Tienda_Aurelion_Dashboard_Sprint2.pbix` o `.pbit`
- **Guía**: `Sprint-2/documentacion/GUIA_RAPIDA_DASHBOARD_POWERBI.md`
- **Requisito**: Microsoft Power BI Desktop (gratis)

### 🌐 Aplicación Web Streamlit

Explora los datos en una aplicación web interactiva:
```bash
# Ejecuta desde la carpeta Sprint-2/
streamlit run programas/app_streamlit.py
```

**Ambas herramientas complementan este análisis con visualizaciones interactivas y funcionalidades adicionales.**

---


---

## 📝 Interpretación de Resultados Orientada al Problema

### Conclusiones Principales:

1. **Distribución de Precios**: Los precios muestran una distribución asimétrica positiva, lo que indica que hay más productos económicos que productos premium. Esto es positivo para la diversificación del catálogo.

2. **Correlaciones Identificadas**: 
   - La correlación entre precio y stock es débil, lo que sugiere que no hay una estrategia clara de mantener más stock de productos económicos.
   - La correlación precio-cantidad vendida ayuda a entender la demanda por segmento de precio.

3. **Outliers**: Los valores extremos identificados pueden representar productos especiales de alto valor o ventas excepcionales que requieren atención especial.

4. **Tendencias de Ventas**: La evolución temporal de ventas permite identificar patrones y planificar mejor el inventario.

### Recomendaciones:

- **Estrategia de Precios**: Considerar ajustar estrategias de stock según la correlación precio-demanda.
- **Gestión de Outliers**: Revisar productos con valores extremos para determinar si son errores o productos especiales.
- **Análisis Temporal**: Profundizar en el análisis de tendencias temporales para mejorar la predicción de demanda.

---

**© 2025 Martos Ludmila - Tienda Aurelion**


---

## 8. 🤖 Machine Learning - Predicción de Ventas <a id='machine-learning'></a>

Complementamos el análisis estadístico con un modelo de Machine Learning para predecir el total de ventas.

### Modelo: Random Forest Regressor
- **Tipo**: Regresión supervisada
- **Árboles**: 100 estimadores
- **División**: 80% train, 20% test
- **Métricas**: R², MAE, RMSE, MAPE


In [ ]:
# Importar librerías de Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("=" * 80)
print("🤖 MACHINE LEARNING - PREDICCIÓN DE VENTAS")
print("=" * 80)


In [ ]:
# Ingeniería de características para ML
print("\n🔧 Preparando datos para Machine Learning...")

# Extraer características temporales de ventas
df_ventas_ml = df_ventas.copy()
df_ventas_ml['mes'] = df_ventas_ml['fecha'].dt.month
df_ventas_ml['dia_semana'] = df_ventas_ml['fecha'].dt.dayofweek
df_ventas_ml['dia_mes'] = df_ventas_ml['fecha'].dt.day

# Unir detalle con productos
df_detalle_ml = df_detalle.merge(
    df_productos[['id', 'categoria', 'precio']], 
    left_on='id_producto', 
    right_on='id'
)

# Calcular características agregadas por venta
caracteristicas = df_detalle_ml.groupby('id_venta').agg({
    'cantidad': 'sum',
    'id_producto': 'nunique',
    'precio_unitario': 'mean',
    'subtotal': 'sum',
    'categoria': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0]
}).reset_index()

caracteristicas.columns = ['id_venta', 'cantidad_total', 'productos_unicos', 
                            'precio_promedio', 'subtotal_calc', 'categoria_principal']

# Unir con ventas
df_ml = df_ventas_ml.merge(caracteristicas, on='id_venta')

# One-Hot Encoding
df_ml = pd.get_dummies(df_ml, columns=['categoria_principal'], prefix='cat')

print(f"✅ Dataset ML preparado: {len(df_ml)} registros, {df_ml.shape[1]} características")


In [ ]:
# Entrenamiento del modelo
print("\n🎓 ENTRENAMIENTO DEL MODELO")
print("-" * 80)

# Preparar X e y
cols_excluir = ['id_venta', 'id_cliente', 'fecha', 'total', 'subtotal_calc']
X = df_ml.drop(columns=cols_excluir)
y = df_ml['total']

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"   Train: {len(X_train)} ({len(X_train)/len(X)*100:.0f}%)")
print(f"   Test: {len(X_test)} ({len(X_test)/len(X)*100:.0f}%)")

# Entrenar modelo
print("\n🌲 Entrenando Random Forest (100 árboles)...")
modelo_rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
modelo_rf.fit(X_train, y_train)
print("✅ Modelo entrenado")


In [ ]:
# Evaluación del modelo
print("\n📊 MÉTRICAS DE EVALUACIÓN")
print("=" * 80)

# Predicciones
y_pred_train = modelo_rf.predict(X_train)
y_pred_test = modelo_rf.predict(X_test)

# Calcular métricas
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mape_test = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

print("\n🎓 ENTRENAMIENTO:")
print(f"   • R² Score: {r2_train:.4f} ({r2_train*100:.2f}%)")
print(f"   • MAE: {mae_train:.2f} monedas")

print("\n🧪 PRUEBA:")
print(f"   • R² Score: {r2_test:.4f} ({r2_test*100:.2f}%)")
print(f"   • MAE: {mae_test:.2f} monedas")
print(f"   • RMSE: {rmse_test:.2f} monedas")
print(f"   • MAPE: {mape_test:.2f}%")

print(f"\n💡 El modelo explica el {r2_test*100:.1f}% de la variabilidad en ventas")


In [ ]:
# Visualizaciones del modelo ML
print("\n📈 GRÁFICOS DEL MODELO ML")
print("-" * 80)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Predicciones vs Reales
axes[0, 0].scatter(y_test, y_pred_test, alpha=0.6, s=80, edgecolors='black', linewidth=0.5)
axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Valor Real (monedas)')
axes[0, 0].set_ylabel('Predicción (monedas)')
axes[0, 0].set_title(f'Predicciones vs Valores Reales (R²={r2_test:.4f})')
axes[0, 0].grid(True, alpha=0.3)

# 2. Distribución de Errores
errores_ml = y_test - y_pred_test
axes[0, 1].hist(errores_ml, bins=15, edgecolor='black', alpha=0.7, color='coral')
axes[0, 1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Error (Real - Predicción)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Errores')
axes[0, 1].grid(True, alpha=0.3)

# 3. Importancia de Características
importancias_ml = pd.DataFrame({
    'caracteristica': X.columns,
    'importancia': modelo_rf.feature_importances_
}).sort_values('importancia', ascending=False).head(10)

colors = plt.cm.viridis(np.linspace(0, 1, len(importancias_ml)))
axes[1, 0].barh(range(len(importancias_ml)), importancias_ml['importancia'], color=colors)
axes[1, 0].set_yticks(range(len(importancias_ml)))
axes[1, 0].set_yticklabels(importancias_ml['caracteristica'])
axes[1, 0].set_xlabel('Importancia')
axes[1, 0].set_title('Top 10 Características Importantes')
axes[1, 0].invert_yaxis()
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Residuos vs Predicciones
axes[1, 1].scatter(y_pred_test, errores_ml, alpha=0.6, s=80, edgecolors='black')
axes[1, 1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Predicción (monedas)')
axes[1, 1].set_ylabel('Residuo')
axes[1, 1].set_title(f'Residuos vs Predicciones (MAE={mae_test:.2f})')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'grafico4_modelo_ml.png'), dpi=300, bbox_inches='tight')
print("✅ Guardado: grafico4_modelo_ml.png")
plt.show()


In [ ]:
# Resumen final con ML incluido
print("\n" + "=" * 80)
print("📊 RESUMEN EJECUTIVO COMPLETO (CON ML)")
print("=" * 80)

print(f"""
📈 ANÁLISIS ESTADÍSTICO:
   • Variables analizadas: Precio, Stock, Total ventas
   • Distribuciones identificadas: {len(distribuciones)} tipos
   • Outliers detectados: {len(outliers_precio) + len(outliers_stock) + len(outliers_total)} totales

🤖 MACHINE LEARNING:
   • Modelo: Random Forest Regressor (100 árboles)
   • R² Score: {r2_test:.4f} ({r2_test*100:.2f}%)
   • MAE: {mae_test:.2f} monedas
   • MAPE: {mape_test:.2f}%

🎯 TOP 3 CARACTERÍSTICAS IMPORTANTES:
""")

for i, (_, row) in enumerate(importancias_ml.head(3).iterrows(), 1):
    print(f"   {i}. {row['caracteristica']}: {row['importancia']*100:.2f}%")

print(f"""
📁 GRÁFICOS GENERADOS:
   1. grafico1_distribucion_precios.png
   2. grafico2_matriz_correlacion.png
   3. grafico3_outliers_ventas.png
   4. grafico4_modelo_ml.png

✅ ANÁLISIS COMPLETADO
""")
print("=" * 80)
